In [73]:
# crear una red neuronal de una capa (perceptron) para predecir sore el dataset wine 
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine
import pandas as pd
import numpy as np
from statsmodels.api import add_constant as ac
from statsmodels.api import OLS, GLM
from statsmodels.formula.api import ols, glm
from statsmodels.tools import add_constant
from statsmodels.datasets import get_rdataset as load_data
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from statsmodels.stats.weightstats import DescrStatsW
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd   
import matplotlib.pyplot as plt
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.diagnostic import lilliefors
from statsmodels.stats.diagnostic import acorr_ljungbox 
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

In [82]:
wine_raw=load_wine()

wine_df=pd.DataFrame(data=wine_raw.data, columns=wine_raw.feature_names)
wine_df

wine_df['target'] = wine_raw.target
display(wine_df)
#Eliminamos las clases que son 2 asi queda binario
wine_df=wine_df[wine_df['target']!=2]
display(wine_df)



,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,12.07,2.16,2.17,21.0,85.0,2.60,2.65,0.37,1.35,2.76,0.86,3.28,378.0,1
126,12.43,1.53,2.29,21.5,86.0,2.74,3.15,0.39,1.77,3.94,0.69,2.84,352.0,1
127,11.79,2.13,2.78,28.5,92.0,2.13,2.24,0.58,1.76,3.00,0.97,2.44,466.0,1
128,12.37,1.63,2.30,24.5,88.0,2.22,2.45,0.40,1.90,2.12,0.89,2.78,342.0,1


In [104]:



X=wine_raw.data
y=wine_raw.target
X=X[y!=2]
y=y[y!=2]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train_p1=np.where(y_train==1,1,-1)
y_test_p1=np.where(y_test==1,1,-1)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)



print(f'Mean Squared Error: {mse(y_test, y_pred)}')
train_accuracy = model.score(X_train, y_train)
test_accuracy = model.score(X_test, y_test)
print(f'Test Accuracy (model.score): {test_accuracy:0.6f}')
print(f'Train Accuracy (model.score): {train_accuracy:0.6f}')


Mean Squared Error: 0.030303030303030304
Test Accuracy (model.score): 0.969697
Train Accuracy (model.score): 1.000000


In [202]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1) ## Se le agrega una dimension porque lo requiere asi pytorch
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

class Percepton(nn.Module):
    def __init__(self, input_dim, p_drop=0):
        super().__init__()
        self.dropout = nn.Dropout(p_drop) ## Dropout no es necesario en una sola capa pero se deja para referencia
        self.layer = nn.Linear(input_dim, 1)
        self.activation = nn.Sigmoid()  ## usamos la funcion sigmoid porque queremos claificar la salida como 0 ó 1
        
    def forward(self, x):
        #x=self.dropout(x)
        x = self.layer(x)
        x = self.activation(x)
        return x
    
input_dim =X_train_tensor.shape[1]
model = Percepton(input_dim)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

epochs = 1000
for epoch in range(epochs):
    outputs = model(X_train_tensor)
    loss=criterion(outputs, y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
with torch.no_grad():
    y_pred_train = model(X_train_tensor)
    y_pred_test = model(X_test_tensor)
    y_pred_train_class = (y_pred_train.numpy() > 0.5).astype(int)
    y_pred_test_class = (y_pred_test.numpy() > 0.5).astype(int)
    train_accuracy = (y_pred_train_class == y_train_tensor.numpy()).mean()
    test_accuracy = (y_pred_test_class == y_test_tensor.numpy()).mean()
    print(f'Test Accuracy: {test_accuracy:.4f}')
    print(f'Train Accuracy: {train_accuracy:.4f}')

Epoch [100/1000], Loss: 0.3731
Epoch [200/1000], Loss: 0.2428
Epoch [300/1000], Loss: 0.1890
Epoch [400/1000], Loss: 0.1582
Epoch [500/1000], Loss: 0.1376
Epoch [600/1000], Loss: 0.1226
Epoch [700/1000], Loss: 0.1111
Epoch [800/1000], Loss: 0.1018
Epoch [900/1000], Loss: 0.0942
Epoch [1000/1000], Loss: 0.0878
Test Accuracy: 0.9394
Train Accuracy: 0.9897


- Implementar la clase MLP3 percepton multicapa de 3 capas densas
- Entrenar y evaluar y compara los resultados para MLP3

In [230]:
## Creamos un clase MLP3 percepton multicapa de 3 capas densas
class MLP3(nn.Module):
    def __init__(self, input_dim, hidden_dim1=16, hidden_dim2=8, p_drop=0.2):
        super().__init__()
        self.dropout = nn.Dropout(p_drop)
        self.layer1 = nn.Linear(input_dim, hidden_dim1)
        self.layer2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.layer3 = nn.Linear(hidden_dim2, 1)
        self.activation = nn.Sigmoid()
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer3(x)
        x = self.activation(x)
        return x
    
input_dim =X_train_tensor.shape[1]
mlp3 = MLP3(input_dim)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(mlp3.parameters(), lr=0.01)

mlp3.train()
epochs = 1000
for epoch in range(epochs):
    outputs = mlp3(X_train_tensor)
    loss=criterion(outputs, y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

mlp3.eval()
with torch.no_grad():
    y_pred_train = mlp3(X_train_tensor)
    y_pred_test = mlp3(X_test_tensor)
    y_pred_train_class = (y_pred_train.numpy() > 0.5).astype(int)
    y_pred_test_class = (y_pred_test.numpy() > 0.5).astype(int)
    train_accuracy = (y_pred_train_class == y_train_tensor.numpy()).mean()
    test_accuracy = (y_pred_test_class == y_test_tensor.numpy()).mean()
    print(f'Test Accuracy: {test_accuracy:.4f}')
    print(f'Train Accuracy: {train_accuracy:.4f}')

Epoch [100/1000], Loss: 0.6324
Epoch [200/1000], Loss: 0.5386
Epoch [300/1000], Loss: 0.4082
Epoch [400/1000], Loss: 0.2698
Epoch [500/1000], Loss: 0.2254
Epoch [600/1000], Loss: 0.1451
Epoch [700/1000], Loss: 0.1367
Epoch [800/1000], Loss: 0.1150
Epoch [900/1000], Loss: 0.0707
Epoch [1000/1000], Loss: 0.0467
Test Accuracy: 0.9697
Train Accuracy: 1.0000


Ahora voy a exportar esta red para poder dibujarla

In [231]:
from torchviz import make_dot

dummy = torch.randn(1, input_dim)
logits = mlp3(dummy)
dot = make_dot(logits, params=dict(mlp3.named_parameters()))
dot.render("mlp3_graph", format="png")  # genera mlp3_graph.png
print("Diagrama guardado en mlp3_graph.png")

Diagrama guardado en mlp3_graph.png


In [232]:
dummy = torch.randn(1, input_dim)
torch.onnx.export(
    mlp3, dummy, "mlp3.onnx",
    input_names=["input"], output_names=["logits"],
    opset_version=17, do_constant_folding=True
)
print("Exportado a mlp3.onnx (abrilo en netron.app)")

Exportado a mlp3.onnx (abrilo en netron.app)


Hacemos otra version

In [233]:
import torch
from torch import nn
from torchviz import make_dot

# --- Modelo ---
class MLP3(nn.Module):
    def __init__(self, input_dim, hidden_dim1=16, hidden_dim2=8, p_drop=0.2):
        super().__init__()
        self.dropout = nn.Dropout(p_drop)
        self.layer1 = nn.Linear(input_dim, hidden_dim1)
        self.layer2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.layer3 = nn.Linear(hidden_dim2, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.dropout(x)
        x = self.layer3(x)              # <-- sin Sigmoid aquí
        return x                        # logits

# --- Datos (asegurate de dtypes/shapes) ---
# X_train_tensor: torch.float32, shape (n_samples, input_dim)
# y_train_tensor: torch.float32, shape (n_samples, 1)
input_dim = X_train_tensor.shape[1]
y_train_tensor = y_train_tensor.to(torch.float32).view(-1, 1)
y_test_tensor  = y_test_tensor.to(torch.float32).view(-1, 1)

mlp3 = MLP3(input_dim)

# Pérdida estable para binario con logits
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(mlp3.parameters(), lr=0.01)

# --- Entrenamiento ---
mlp3.train()
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    logits = mlp3(X_train_tensor)
    loss = criterion(logits, y_train_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}]  Loss: {loss.item():.4f}')

# --- Evaluación ---
mlp3.eval()
with torch.no_grad():
    # Probabilidades = sigmoid(logits)
    y_pred_train_prob = torch.sigmoid(mlp3(X_train_tensor))
    y_pred_test_prob  = torch.sigmoid(mlp3(X_test_tensor))

    y_pred_train_class = (y_pred_train_prob > 0.5).int()
    y_pred_test_class  = (y_pred_test_prob  > 0.5).int()

    train_accuracy = (y_pred_train_class.cpu().numpy() == y_train_tensor.cpu().numpy()).mean()
    test_accuracy  = (y_pred_test_class.cpu().numpy()  == y_test_tensor.cpu().numpy()).mean()

    print(f'Test Accuracy:  {test_accuracy:.4f}')
    print(f'Train Accuracy: {train_accuracy:.4f}')

# --- Graficar con torchviz ---
# Usamos un batch dummy (puede ser un batch real también)
dummy = torch.randn(1, input_dim)
logits = mlp3(dummy)
dot = make_dot(logits, params=dict(mlp3.named_parameters()))
dot.render("mlp3_graph", format="png")  # genera mlp3_graph.png
print("Diagrama guardado en mlp3_graph_2.png")


Epoch [100/1000]  Loss: 0.6784
Epoch [200/1000]  Loss: 0.6498
Epoch [300/1000]  Loss: 0.5992
Epoch [400/1000]  Loss: 0.5383
Epoch [500/1000]  Loss: 0.4786
Epoch [600/1000]  Loss: 0.4185
Epoch [700/1000]  Loss: 0.3519
Epoch [800/1000]  Loss: 0.2687
Epoch [900/1000]  Loss: 0.3126
Epoch [1000/1000]  Loss: 0.2647
Test Accuracy:  0.9394
Train Accuracy: 1.0000
Diagrama guardado en mlp3_graph_2.png


In [ ]:
## crear un percepton con pytorch

class Perceptron2(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )
        self.fc = nn.Linear(input_dim, output_dim)
        
        
    def forward(self, x):
        return self.fc(x)